In [13]:

import pandas as pd
import numpy as np
from collections import defaultdict
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 160)


In [15]:
df = pd.read_excel('/content/Week_routes.xlsx',sheet_name="Sheet1")

In [16]:
# Clean headers
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(r"[^a-z0-9_]", "", regex=True)
)

In [20]:
# Known renames
df = df.rename(columns={
    "trip_finalkms": "trip_final_kms",
    "daynight": "day_night",
})

expected = [
    "trip_date","trip_time","day_night","route_no","tripsheet_no",
    "zone","distance","cab_type","latitude","longitude","area","facility","city"
]
missing = [c for c in expected if c not in df.columns]
if missing:
    print("⚠️ Missing expected columns:", missing)
print(df.shape, df.columns.tolist())
df.head()

(51689, 16) ['city', 'facility', 'trip_date', 'day_night', 'trip_time', 'travel_type_id', 'tripsheet_no', 'route_no', 'area', 'trip_final_kms', 'cab_no', 'cab_type', 'zone', 'distance', 'latitude', 'longitude']


,city,facility,trip_date,day_night,trip_time,travel_type_id,tripsheet_no,route_no,area,trip_final_kms,cab_no,cab_type,zone,distance,latitude,longitude
0,BANGALORE,BLR-MBP,2019-01-26,Night,04:00:00,IN,14623830,1001,WHITEFIELD - BIG BAZAAR,44.0,KA53C7195,SWIFT DZIER (4 Seater),7,22.0,12.988019,77.731276
1,BANGALORE,BLR-MBP,2019-01-26,Night,05:00:00,IN,14623917,1024,T C PALAYA - KRISTU JYOTHI COLLEGE,34.0,KA437197,SWIFT DZIER (4 Seater),32,16.0,13.021354,77.704029
2,BANGALORE,BLR-MBP,2019-01-26,Night,05:00:00,IN,14623917,1024,T C PALAYA - KRISTU JYOTHI COLLEGE,34.0,KA437197,SWIFT DZIER (4 Seater),32,16.0,13.021354,77.704029
3,BANGALORE,BLR-MBP,2019-01-26,Night,05:00:00,IN,14623917,1024,T C PALAYA - KRISTU JYOTHI COLLEGE,34.0,KA437197,SWIFT DZIER (4 Seater),32,16.0,13.021354,77.704029
4,BANGALORE,BLR-MBP,2019-01-26,Night,05:00:00,IN,14623961,1006,NELAGADARANAHALLI - BUS STOP,52.0,KA50A6459,SWIFT DZIER (4 Seater),80,26.0,13.031554,77.503346


In [21]:
# Robust time & date parsing
df["zone"] = pd.to_numeric(df["zone"], errors="coerce")
df["distance"] = pd.to_numeric(df["distance"], errors="coerce")
df["trip_time"] = pd.to_datetime(df["trip_time"].astype(str), errors="coerce").dt.time
df["trip_date"] = pd.to_datetime(df["trip_date"], errors="coerce").dt.date

/tmp/ipython-input-3506447987.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["trip_time"] = pd.to_datetime(df["trip_time"].astype(str), errors="coerce").dt.time


In [11]:
df.columns

Index(['unnamed_0', 'unnamed_1'], dtype='object')

In [22]:
# Numerics
df["zone"] = pd.to_numeric(df["zone"], errors="coerce")
df["distance"] = pd.to_numeric(df["distance"], errors="coerce")

In [61]:
# Learnability sanity check: can we predict actual tripsheet (composite) from coords+zone+distance?
# Uses KNN per (day, shift, IN/OUT). Reports CV accuracy and route-count gap vs actual.

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# --- 0) Normalize raw columns & build composite actual_route_id from tripsheet (+ facility/vendor if available)
dfx = df.copy()

def norm(s): return s.strip().lower().replace(" ", "_")
dfx.columns = [norm(c) for c in dfx.columns]

# Column picks
c_trip_date   = "trip_date"      if "trip_date" in dfx.columns else "tripdate"
c_trip_time   = "trip_time"      if "trip_time" in dfx.columns else None
c_day_night   = "day_night"      if "day_night" in dfx.columns else None
c_tt          = "travel_type_id" if "travel_type_id" in dfx.columns else None
c_zone        = "zone"           if "zone" in dfx.columns else None
c_lat         = "latitude"       if "latitude" in dfx.columns else None
c_lon         = "longitude"      if "longitude" in dfx.columns else None
c_dist        = "distance"       if "distance" in dfx.columns else None
c_route_no    = "route_no"       if "route_no" in dfx.columns else None
c_tripno      = "tripsheet_no"   if "tripsheet_no" in dfx.columns else None
c_vendor      = "vendor"         if "vendor" in dfx.columns else None
c_facility    = "facility"       if "facility" in dfx.columns else None

# Parse types
dfx[c_trip_date] = pd.to_datetime(dfx[c_trip_date], errors="coerce").dt.date
if c_trip_time:
    dfx[c_trip_time] = pd.to_datetime(dfx[c_trip_time].astype(str), errors="coerce").dt.time
if c_day_night and c_trip_time:
    dfx["shift_key"] = dfx[c_day_night].astype(str).str.strip() + "_" + dfx[c_trip_time].astype(str)
elif c_day_night:
    dfx["shift_key"] = dfx[c_day_night].astype(str).str.strip()
else:
    dfx["shift_key"] = "NA"
if c_tt:
    dfx["travel_type_id"] = (dfx[c_tt].astype(str).str.upper().str.strip()
                             .replace({"INWARD":"IN","OUTWARD":"OUT","IN-TRIP":"IN","OUT-TRIP":"OUT"}))
else:
    dfx["travel_type_id"] = "NA"

# Composite actual route id (what ops considers a single trip)
comp = dfx[c_tripno].astype(str)
if c_vendor   and c_vendor   in dfx.columns: comp = comp + "|V=" + dfx[c_vendor].astype(str)
if c_facility and c_facility in dfx.columns: comp = comp + "|F=" + dfx[c_facility].astype(str)
dfx["actual_route_id"] = comp

# Choose a busy day automatically (or set manually)
day = dfx[c_trip_date].value_counts().idxmax()
# day = pd.to_datetime("2019-01-30").date  # uncomment to force

day_df = dfx[dfx[c_trip_date] == day].copy()

# Guard: need features
feat_cols = []
for col in [c_lat, c_lon, c_zone, c_dist]:
    if col and col in day_df.columns:
        feat_cols.append(col)
if not feat_cols or day_df[feat_cols].isna().any().any():
    day_df = day_df.dropna(subset=feat_cols).copy()

print(f"Day selected: {day} | Rows: {len(day_df)} | Features: {feat_cols}")

# --- 1) Per-shift × type evaluation using KNN
results = []
for (shift, ttype), sdf in day_df.groupby(["shift_key", "travel_type_id"]):
    sdf = sdf.dropna(subset=["actual_route_id"] + feat_cols).copy()
    if sdf["actual_route_id"].nunique() < 2 or len(sdf) < 50:
        continue  # not enough classes or data

    X = sdf[feat_cols].to_numpy()
    y = sdf["actual_route_id"].astype(str).to_numpy()

    # Stratified 5-fold CV on employees
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # KNN pipeline with scaling
    pipe = Pipeline([
        ("scaler", StandardScaler(with_mean=False) if {c for c in feat_cols} & {c_zone} else StandardScaler()),
        ("knn", KNeighborsClassifier(n_neighbors=5))
    ])
    try:
        acc = cross_val_score(pipe, X, y, cv=cv, scoring="accuracy").mean()
    except Exception as e:
        print(f"Skip {shift} {ttype} due to error: {e}")
        continue

    # Route-count realism: fit on full, predict, compare unique predicted vs actual (rough proxy)
    pipe.fit(X, y)
    yhat = pipe.predict(X)
    actual_routes = len(pd.unique(y))
    predicted_routes = len(pd.unique(yhat))
    route_gap = predicted_routes - actual_routes
    pct_gap = (route_gap / actual_routes * 100.0) if actual_routes else np.nan

    results.append({
        "shift_key": shift,
        "travel_type": ttype,
        "rows": len(sdf),
        "actual_routes": actual_routes,
        "predicted_routes_sim": predicted_routes,
        "route_count_gap": route_gap,
        "route_pct_gap": round(pct_gap, 2) if actual_routes else np.nan,
        "cv_accuracy": round(acc, 3)
    })

res_df = pd.DataFrame(results).sort_values(["shift_key","travel_type"])
print("Learnability check — per shift × type")
display(res_df)

# Quick day summary
if not res_df.empty:
    print("\nDay-level summary:")
    print("Avg CV accuracy:", round(res_df["cv_accuracy"].mean(), 3))
    print("Median route % gap:", round(res_df["route_pct_gap"].median(), 2), "%")


/tmp/ipython-input-1028462926.py:34: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dfx[c_trip_time] = pd.to_datetime(dfx[c_trip_time].astype(str), errors="coerce").dt.time


Day selected: 2019-01-30 | Rows: 10075 | Features: ['latitude', 'longitude', 'zone', 'distance']
Skip Day_14:00:00 OUT due to error: n_splits=5 cannot be greater than the number of members in each class.


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklea

Skip Night_05:00:00 IN due to error: n_splits=5 cannot be greater than the number of members in each class.
Skip Night_20:30:00 IN due to error: n_splits=5 cannot be greater than the number of members in each class.


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Learnability check — per shift × type


,shift_key,travel_type,rows,actual_routes,predicted_routes_sim,route_count_gap,route_pct_gap,cv_accuracy
0,Day_08:00:00,IN,292,78,59,-19,-24.36,0.480
1,Day_11:30:00,IN,624,114,90,-24,-21.05,0.620
2,Day_12:30:00,IN,435,100,76,-24,-24.00,0.522
3,Day_13:30:00,IN,373,79,57,-22,-27.85,0.563
4,Day_14:30:00,IN,297,81,54,-27,-33.33,0.505
5,Day_15:00:00,OUT,118,44,26,-18,-40.91,0.355
6,Day_16:00:00,IN,563,106,81,-25,-23.58,0.574
7,Day_16:00:00,OUT,330,86,65,-21,-24.42,0.479
8,Day_17:00:00,OUT,269,84,53,-31,-36.90,0.487
9,Day_17:30:00,IN,906,141,110,-31,-21.99,0.682



Day-level summary:
Avg CV accuracy: 0.499
Median route % gap: -31.58 %


In [23]:
# Capacity hint from cab_type (for analysis only)
def extract_capacity(cab_type):
    if pd.isna(cab_type): return np.nan
    s = str(cab_type).lower()
    for cap in (12, 6, 4):
        if str(cap) in s: return cap
    return np.nan

df["actual_capacity_hint"] = df["cab_type"].apply(extract_capacity)

# Shift key
df["day_night"] = df["day_night"].astype(str).str.strip()
df["shift_key"] = df["day_night"] + "_" + df["trip_time"].astype(str)

print("Rows after coercions:", len(df))
df[["trip_date","trip_time","day_night","zone","distance","actual_capacity_hint","shift_key"]].head(8)

Rows after coercions: 51689


,trip_date,trip_time,day_night,zone,distance,actual_capacity_hint,shift_key
0,2019-01-26,04:00:00,Night,7.0,22.0,4.0,Night_04:00:00
1,2019-01-26,05:00:00,Night,32.0,16.0,4.0,Night_05:00:00
2,2019-01-26,05:00:00,Night,32.0,16.0,4.0,Night_05:00:00
3,2019-01-26,05:00:00,Night,32.0,16.0,4.0,Night_05:00:00
4,2019-01-26,05:00:00,Night,80.0,26.0,4.0,Night_05:00:00
5,2019-01-26,05:00:00,Night,129.0,23.0,4.0,Night_05:00:00
6,2019-01-26,05:00:00,Night,127.0,22.0,4.0,Night_05:00:00
7,2019-01-26,05:00:00,Night,1.0,18.0,4.0,Night_05:00:00


In [24]:
# Step 1: Sort by Shift → Zone ↑ → Distance ↓
df_sorted = df.sort_values(
    ["shift_key", "zone", "distance"],
    ascending=[True, True, False]
).reset_index(drop=True)

print("Sorted preview:")
df_sorted[["shift_key","zone","distance","area","route_no"]].head(12)


Sorted preview:


,shift_key,zone,distance,area,route_no
0,Day_08:00:00,1.0,36.0,YELAHANKA -PROVIDENT WELWORTH CITY,1002
1,Day_08:00:00,1.0,29.0,RAJANKUNTE - BUS STOP,1005
2,Day_08:00:00,1.0,18.0,YELAHANKA - ATTUR LAYOUT LAST BUS STOP,1003
3,Day_08:00:00,1.0,18.0,YELAHANKA - 4TH PHASE BUS DEPOT,1003
4,Day_08:00:00,1.0,18.0,YELAHANKA - ATTUR LAYOUT LAST BUS STOP,1004
5,Day_08:00:00,1.0,18.0,YELAHANKA - 4TH PHASE BUS DEPOT,1004
6,Day_08:00:00,1.0,18.0,YELAHANKA - 4TH PHASE BUS DEPOT,1005
7,Day_08:00:00,1.0,18.0,YELAHANKA - 4TH PHASE BUS DEPOT,1004
8,Day_08:00:00,1.0,17.0,YELAHANKA - MOTHER DAIRY CROSS,1004
9,Day_08:00:00,1.0,14.0,YELAHANKA - NEW TOWN BUS STOP,1001


In [28]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 4.6 MB/s eta 0:00:00


In [40]:
import pandas as pd
import numpy as np

# --- 0) Clean headers (safe to re-run) ---
df.columns = (
    df.columns.str.strip().str.lower()
      .str.replace(" ", "_")
      .str.replace(r"[^a-z0-9_]", "", regex=True)
)
df = df.rename(columns={"trip_finalkms": "trip_final_kms", "daynight": "day_night"})

# --- 1) Coerce types & keys ---
df["trip_date"] = pd.to_datetime(df["trip_date"], errors="coerce").dt.date
df["trip_time"] = pd.to_datetime(df["trip_time"].astype(str), errors="coerce").dt.time
df["zone"]       = pd.to_numeric(df["zone"], errors="coerce")
df["distance"]   = pd.to_numeric(df["distance"], errors="coerce")
df["day_night"]  = df["day_night"].astype(str).str.strip()

# normalize travel type to IN/OUT
df["travel_type_id"] = (
    df["travel_type_id"].astype(str).str.upper().str.strip()
      .replace({"INWARD":"IN","OUTWARD":"OUT","IN-TRIP":"IN","OUT-TRIP":"OUT"})
)
df["travel_type_id"] = df["travel_type_id"].where(df["travel_type_id"].isin(["IN","OUT"]), other=np.nan)

df["shift_key"] = df["day_night"] + "_" + df["trip_time"].astype(str)

# --- 2) Sort: DATE → SHIFT → TRAVEL_TYPE → ZONE ↑ → DISTANCE ↓ ---
df_sorted = df.sort_values(
    ["trip_date","shift_key","travel_type_id","zone","distance"],
    ascending=[True, True, True, True, False]
).reset_index(drop=True)

# --- 3) Seater packing per (DATE, SHIFT, TRAVEL_TYPE, ZONE) ---
SEATERS = [12, 6, 4]

def assign_pred_routes(group_df):
    if group_df.empty:
        out = group_df.copy()
        for c in ["pred_route_id","pred_capacity","pred_occupancy"]:
            out[c] = []
        return out

    n = len(group_df)
    assignments = np.empty(n, dtype=object)
    pred_routes, remaining = [], list(range(n))

    while remaining:
        left = len(remaining)
        cap = next((c for c in SEATERS if left >= c), None)
        if cap is None:
            # try merging tails into existing space; else partial 4
            placed_any = False
            for idx_local in remaining[:]:
                for r in sorted(pred_routes, key=lambda x: (x["cap"]-len(x["members"])), reverse=True):
                    if len(r["members"]) < r["cap"]:
                        r["members"].append(idx_local)
                        remaining.remove(idx_local)
                        placed_any = True
                        break
            if not placed_any:
                take = min(4, len(remaining))
                pred_routes.append({"cap":4, "members":remaining[:take]})
                remaining = remaining[take:]
            continue
        pred_routes.append({"cap":cap, "members":remaining[:cap]})
        remaining = remaining[cap:]

    base = (f"{group_df['trip_date'].iloc[0]}_{group_df['shift_key'].iloc[0]}"
            f"_{group_df['travel_type_id'].iloc[0]}_Z{int(group_df['zone'].iloc[0])}")
    for i, r in enumerate(pred_routes, start=1):
        rid = f"{base}_R{i:03d}"
        for idx_local in r["members"]:
            assignments[idx_local] = rid

    cap_map, occ_map = {}, {}
    for r in pred_routes:
        rid = assignments[r["members"][0]]
        cap_map[rid] = r["cap"]
        occ_map[rid] = len(r["members"])

    out = group_df.copy()
    out["pred_route_id"]   = assignments
    out["pred_capacity"]   = [cap_map[a] for a in assignments]
    out["pred_occupancy"]  = [occ_map[a] for a in assignments]
    return out

pred_chunks = []
for keys, g in df_sorted.groupby(["trip_date","shift_key","travel_type_id","zone"], sort=False):
    pred_chunks.append(assign_pred_routes(g.reset_index(drop=True)))
pred_df = pd.concat(pred_chunks, ignore_index=True)

# --- 4) History-aware tail alignment within SAME (DATE, SHIFT, TRAVEL_TYPE, ZONE) ---
route_stats = (
    pred_df.groupby("pred_route_id")
    .agg(pred_capacity=("pred_capacity","first"),
         pred_occupancy=("pred_route_id","count"),
         trip_date=("trip_date","first"),
         shift_key=("shift_key","first"),
         travel_type_id=("travel_type_id","first"),
         zone=("zone","first"))
    .reset_index()
)

# initial majority actual route per predicted route
route_actual_counts = (
    pred_df.groupby(["pred_route_id","route_no"]).size()
    .rename("cnt").reset_index()
)
majority_init = (
    route_actual_counts.sort_values(["pred_route_id","cnt"], ascending=[True, False])
    .groupby("pred_route_id").head(1)
    .rename(columns={"route_no":"majority_actual_route","cnt":"majority_count"})
)
maj_map = dict(zip(majority_init["pred_route_id"], majority_init["majority_actual_route"]))

pred_df_hist = pred_df.copy()
cap_map = dict(zip(route_stats["pred_route_id"], route_stats["pred_capacity"]))
occ_map = dict(zip(route_stats["pred_route_id"], route_stats["pred_occupancy"]))

def candidates_for(emp_row):
    d  = emp_row["trip_date"]
    s  = emp_row["shift_key"]
    tt = emp_row["travel_type_id"]
    z  = emp_row["zone"]
    actual = emp_row["route_no"]
    scope_routes = route_stats[
        (route_stats["trip_date"] == d) &
        (route_stats["shift_key"] == s) &
        (route_stats["travel_type_id"] == tt) &
        (route_stats["zone"] == z)
    ]["pred_route_id"].tolist()
    cands = []
    for rid in scope_routes:
        if rid == emp_row["pred_route_id"]:
            continue
        maj = maj_map.get(rid, None)
        if maj is None:
            continue
        if maj == actual and occ_map.get(rid,0) < cap_map.get(rid,0):
            cands.append(rid)
    cands.sort(key=lambda r: (cap_map[r] - occ_map[r]), reverse=True)
    return cands

tail_ids = route_stats.loc[route_stats["pred_occupancy"] < 4, "pred_route_id"].tolist()
moved = 0
for rid in tail_ids:
    idxs = pred_df_hist.index[pred_df_hist["pred_route_id"] == rid].tolist()
    idxs = sorted(idxs, key=lambda ix: pred_df_hist.at[ix,"distance"], reverse=True)  # far→near
    for ix in idxs:
        emp = pred_df_hist.loc[ix]
        for target in candidates_for(emp):
            if occ_map[target] < cap_map[target]:
                pred_df_hist.at[ix, "pred_route_id"] = target
                occ_map[target] += 1
                occ_map[rid]     -= 1
                moved += 1
                break  # next employee

# recompute occupancy & majority after moves
pred_df_hist["pred_occupancy"] = pred_df_hist.groupby("pred_route_id")["pred_route_id"].transform("count")
route_actual_counts2 = (
    pred_df_hist.groupby(["pred_route_id","route_no"]).size()
    .rename("cnt").reset_index()
)
majority2 = (
    route_actual_counts2.sort_values(["pred_route_id","cnt"], ascending=[True, False])
    .groupby("pred_route_id").head(1)
    .rename(columns={"route_no":"majority_actual_route","cnt":"majority_count"})
)
maj_map2 = dict(zip(majority2["pred_route_id"], majority2["majority_actual_route"]))

aligned_df = pred_df_hist.copy()
aligned_df["majority_actual_for_pred"] = aligned_df["pred_route_id"].map(maj_map2)
aligned_df["is_correct_route_match"]   = aligned_df["route_no"] == aligned_df["majority_actual_for_pred"]

# --- 5) Tripsheet numbers (unique per predicted route) ---
rng = np.random.default_rng(2025)
unique_routes = aligned_df["pred_route_id"].drop_duplicates().sort_values().tolist()
tripnos = {rid: int(rng.integers(7_000_000, 9_999_999)) for rid in unique_routes}
aligned_df["pred_tripsheet_no"] = aligned_df["pred_route_id"].map(tripnos)

# --- 6) Validation (capacity, zone, travel_type purity) ---
occ_vs_cap = (
    aligned_df.groupby("pred_route_id")
    .agg(capacity=("pred_capacity","first"),
         occupancy=("pred_route_id","count"))
    .reset_index()
)
occ_vs_cap["over_capacity"] = occ_vs_cap["occupancy"] > occ_vs_cap["capacity"]

zone_misalignment = (
    aligned_df.groupby("pred_route_id")["zone"].nunique().reset_index(name="distinct_zones_in_route")
)
zone_misalignment["is_zone_misaligned"] = zone_misalignment["distinct_zones_in_route"] > 1

tt_misalignment = (
    aligned_df.groupby("pred_route_id")["travel_type_id"].nunique().reset_index(name="distinct_travel_types_in_route")
)
tt_misalignment["is_traveltype_misaligned"] = tt_misalignment["distinct_travel_types_in_route"] > 1

validation_report = (occ_vs_cap
    .merge(zone_misalignment, on="pred_route_id", how="left")
    .merge(tt_misalignment, on="pred_route_id", how="left")
)[["pred_route_id","capacity","occupancy","over_capacity",
   "distinct_zones_in_route","is_zone_misaligned",
   "distinct_travel_types_in_route","is_traveltype_misaligned"]].sort_values("pred_route_id")

# --- 7) Metrics, summary, export ---
total_rows  = len(aligned_df)
overall_acc = aligned_df["is_correct_route_match"].sum() / total_rows if total_rows else 0.0
route_purity = majority2["majority_count"].sum() / total_rows if total_rows else 0.0

summary = (
    aligned_df.groupby(["trip_date","shift_key","travel_type_id","zone","pred_route_id","pred_tripsheet_no"])
    .agg(pred_capacity=("pred_capacity","first"),
         occupancy=("pred_route_id","count"),
         majority_actual=("majority_actual_for_pred","first"),
         purity_count=("is_correct_route_match","sum"))
    .reset_index()
)
summary["purity_pct"] = (summary["purity_count"] / summary["occupancy"]).round(3)

print(f"History-aware tail moves applied: {moved}")
print("Any over-capacity?            ", bool(validation_report["over_capacity"].any()))
print("Any zone mix?                 ", bool(validation_report["is_zone_misaligned"].any()))
print("Any IN/OUT mix in a route?    ", bool(validation_report["is_traveltype_misaligned"].any()))
print(f"Overall employee accuracy:     {overall_acc:.4f}")
print(f"Route purity (weighted):       {route_purity:.4f}")

out_path = "predicted_routes_by_date_shift_type_historyAware.xlsx"
with pd.ExcelWriter(out_path, engine="xlsxwriter") as w:
    df_sorted.to_excel(w, index=False, sheet_name="01_sorted")
    aligned_df.to_excel(w, index=False, sheet_name="02_predicted_routes_hist")
    validation_report.to_excel(w, index=False, sheet_name="03_validation")
    summary.to_excel(w, index=False, sheet_name="04_summary")
    pd.DataFrame({
        "metric": ["employee_accuracy","route_purity","tail_moves"],
        "value": [round(overall_acc,4), round(route_purity,4), int(moved)]
    }).to_excel(w, index=False, sheet_name="05_metrics")

print("Saved:", out_path)


/tmp/ipython-input-2521973756.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["trip_time"] = pd.to_datetime(df["trip_time"].astype(str), errors="coerce").dt.time


History-aware tail moves applied: 0
Any over-capacity?             False
Any zone mix?                  False
Any IN/OUT mix in a route?     False
Overall employee accuracy:     0.8403
Route purity (weighted):       0.8403
Saved: predicted_routes_by_date_shift_type_historyAware.xlsx


In [59]:
# # 🧩 Micro-tail consolidation using Week_Routes patterns (ONLY occupancy <= 2)
# # Scope: SAME (trip_date, shift_key, travel_type_id, zone); target route must match employee's actual route_no (majority)
# # Safe: no over-capacity; no IN/OUT/zone/date/shift mixing.

# import pandas as pd
# import numpy as np

# dfm = aligned_df.copy()

# # 1) Build route stats and initial majority actual per predicted route
# route_stats = (
#     dfm.groupby("pred_route_id")
#        .agg(pred_capacity=("pred_capacity","first"),
#             pred_occupancy=("pred_route_id","count"),
#             trip_date=("trip_date","first"),
#             shift_key=("shift_key","first"),
#             travel_type_id=("travel_type_id","first"),
#             zone=("zone","first"))
#        .reset_index()
# )

# route_actual_counts = (
#     dfm.groupby(["pred_route_id","route_no"]).size()
#        .rename("cnt").reset_index()
# )

# majority = (
#     route_actual_counts.sort_values(["pred_route_id","cnt"], ascending=[True, False])
#                        .groupby("pred_route_id").head(1)
#                        .rename(columns={"route_no":"majority_actual_route","cnt":"majority_count"})
# )

# maj_map = dict(zip(majority["pred_route_id"], majority["majority_actual_route"]))

# # 2) Identify micro-tail routes (occupancy <= 2)
# micro_ids = route_stats.loc[route_stats["pred_occupancy"] <= 2, "pred_route_id"].tolist()
# if not micro_ids:
#     print("No micro-tail routes (<=2 occupancy) found. Nothing to consolidate.")
# else:
#     print(f"Micro-tail routes found: {len(micro_ids)}")

# # 3) Live occupancy maps
# cap_map = dict(zip(route_stats["pred_route_id"], route_stats["pred_capacity"]))
# occ_map = dict(zip(route_stats["pred_route_id"], route_stats["pred_occupancy"]))

# # Helper: find candidate routes within same (date, shift, type, zone) whose majority matches employee's actual route
# def history_matched_candidates(emp_row):
#     d  = emp_row["trip_date"]
#     s  = emp_row["shift_key"]
#     tt = emp_row["travel_type_id"]
#     z  = emp_row["zone"]
#     actual = emp_row["route_no"]

#     scope_ids = route_stats[
#         (route_stats["trip_date"] == d) &
#         (route_stats["shift_key"] == s) &
#         (route_stats["travel_type_id"] == tt) &
#         (route_stats["zone"] == z)
#     ]["pred_route_id"].tolist()

#     cands = []
#     for rid in scope_ids:
#         if rid == emp_row["pred_route_id"]:
#             continue
#         maj = maj_map.get(rid, None)
#         if maj is None:
#             continue
#         if maj == actual and occ_map.get(rid, 0) < cap_map.get(rid, 0):
#             cands.append(rid)

#     # Prefer most free seats first, then (optional) larger capacity
#     cands.sort(key=lambda r: (cap_map[r] - occ_map[r], cap_map[r]), reverse=True)
#     return cands

# # 4) Move employees out of micro-tail routes, far→near (keep sequence logic)
# moved = 0
# for rid in micro_ids:
#     idxs = dfm.index[dfm["pred_route_id"] == rid].tolist()
#     # process farthest first to preserve far→near pickup shape
#     idxs = sorted(idxs, key=lambda ix: dfm.at[ix, "distance"], reverse=True)
#     for ix in idxs:
#         emp = dfm.loc[ix]
#         for target in history_matched_candidates(emp):
#             if occ_map[target] < cap_map[target]:
#                 dfm.at[ix, "pred_route_id"] = target
#                 occ_map[target] += 1
#                 occ_map[rid]     -= 1
#                 moved += 1
#                 break  # next employee

# # 5) Recompute occupancies & majorities; refresh flags
# dfm["pred_occupancy"] = dfm.groupby("pred_route_id")["pred_route_id"].transform("count")

# route_actual_counts2 = (
#     dfm.groupby(["pred_route_id","route_no"]).size()
#        .rename("cnt").reset_index()
# )
# majority2 = (
#     route_actual_counts2.sort_values(["pred_route_id","cnt"], ascending=[True, False])
#                         .groupby("pred_route_id").head(1)
#                         .rename(columns={"route_no":"majority_actual_route","cnt":"majority_count"})
# )
# maj_map2 = dict(zip(majority2["pred_route_id"], majority2["majority_actual_route"]))
# dfm["majority_actual_for_pred"] = dfm["pred_route_id"].map(maj_map2)
# dfm["is_correct_route_match"]   = dfm["route_no"] == dfm["majority_actual_for_pred"]

# # 6) Quick validations (no over-capacity, no IN/OUT mix, no zone mix)
# occ_vs_cap = (
#     dfm.groupby("pred_route_id")
#        .agg(capacity=("pred_capacity","first"),
#             occupancy=("pred_route_id","count"))
#        .reset_index()
# )
# over_capacity = bool((occ_vs_cap["occupancy"] > occ_vs_cap["capacity"]).any())

# zone_mix = (
#     dfm.groupby("pred_route_id")["zone"].nunique().reset_index(name="dz")
# )
# type_mix = (
#     dfm.groupby("pred_route_id")["travel_type_id"].nunique().reset_index(name="dt")
# )
# zone_misaligned = bool((zone_mix["dz"] > 1).any())
# tt_misaligned   = bool((type_mix["dt"] > 1).any())

# print(f"Micro-tail employees moved: {moved}")
# print("Any over-capacity?         ", over_capacity)
# print("Any zone mix?              ", zone_misaligned)
# print("Any IN/OUT mix in a route? ", tt_misaligned)

# # 7) Replace aligned_df with improved version and (optionally) reassign fresh tripsheet numbers
# aligned_df = dfm.copy()

# # If you want NEW random tripsheet numbers per predicted route after consolidation, uncomment below:
# # rng = np.random.default_rng(2025)
# # unique_routes = aligned_df["pred_route_id"].drop_duplicates().sort_values().tolist()
# # tripnos = {rid: int(rng.integers(7_000_000, 9_999_999)) for rid in unique_routes}
# # aligned_df["pred_tripsheet_no"] = aligned_df["pred_route_id"].map(tripnos)

# # 8) (Optional) quick delta on total predicted routes by day after consolidation
# pred_daily_before = (
#     route_stats.groupby("trip_date")["pred_route_id"].count().rename("pred_routes_before")
#     .reset_index()
# )
# pred_daily_after = (
#     aligned_df.groupby(["trip_date","shift_key","travel_type_id"])["pred_tripsheet_no"]
#               .nunique().reset_index(name="pred_routes_after")
#               .groupby("trip_date")["pred_routes_after"].sum().reset_index()
# )
# pred_delta = pred_daily_before.merge(pred_daily_after, on="trip_date", how="outer").fillna(0)
# pred_delta["reduction"] = pred_delta["pred_routes_before"] - pred_delta["pred_routes_after"]
# print("\nPredicted routes delta by day (before vs after micro-tail consolidation):")
# display(pred_delta)


Micro-tail routes found: 8165
Micro-tail employees moved: 0
Any over-capacity?          False
Any zone mix?               False
Any IN/OUT mix in a route?  False

Predicted routes delta by day (before vs after micro-tail consolidation):


,trip_date,pred_routes_before,pred_routes_after,reduction
0,2019-01-26,1297,1297,0
1,2019-01-27,508,508,0
2,2019-01-28,2241,2241,0
3,2019-01-29,2892,2892,0
4,2019-01-30,2936,2936,0
5,2019-01-31,2887,2886,1
6,2019-02-01,2831,2831,0


In [41]:
# Pick day, shift, and travel type explicitly
dfv = aligned_df.copy()
dfv["trip_date_str"] = pd.to_datetime(dfv["trip_date"]).dt.strftime("%Y-%m-%d")

day   = dfv["trip_date_str"].value_counts().idxmax()
shift = dfv.loc[dfv["trip_date_str"]==day, "shift_key"].value_counts().idxmax()
ttype = "IN"  # or "OUT"

scope = dfv[(dfv["trip_date_str"]==day) & (dfv["shift_key"]==shift) & (dfv["travel_type_id"]==ttype)].copy()
print(f"Using DAY={day}, SHIFT={shift}, TYPE={ttype}")
print(scope.groupby("pred_capacity")["pred_route_id"].nunique())

# then sample per seater from `scope` and build maps as before


Using DAY=2019-01-30, SHIFT=Day_17:30:00, TYPE=IN
pred_capacity
4     106
6      44
12     33
Name: pred_route_id, dtype: int64


In [42]:
import os, random, zipfile, pandas as pd
from datetime import datetime

random.seed(22)

# 1) Build safe date string
dfv = aligned_df.copy()
dfv["trip_date_str"] = pd.to_datetime(dfv["trip_date"]).dt.strftime("%Y-%m-%d")

# ---- Choose scope ----
day   = dfv["trip_date_str"].value_counts().idxmax()   # auto: busiest day
shift = dfv.loc[dfv["trip_date_str"]==day, "shift_key"].value_counts().idxmax()
ttype = "IN"   # 🔄 change to "OUT" if you want drop trips

scope = dfv[(dfv["trip_date_str"]==day) &
            (dfv["shift_key"]==shift) &
            (dfv["travel_type_id"]==ttype)].copy()

print(f"Using DAY={day}, SHIFT={shift}, TYPE={ttype}")
print("Routes by seater in this scope:\n",
      scope.groupby("pred_capacity")["pred_route_id"].nunique())

# 2) Pick 1 route per seater
seaters = [4,6,12]
picked = []
for cap in seaters:
    ids = scope.loc[scope["pred_capacity"]==cap,"pred_route_id"].drop_duplicates().tolist()
    if ids:
        picked.append((cap, random.choice(ids)))

# 3) Build maps
OUT_DIR = f"map_validation_{day}_{shift}_{ttype}".replace(":","_")
os.makedirs(OUT_DIR, exist_ok=True)
colors = ["blue","purple","orange"]
html_files = []

for i, (cap, rid) in enumerate(picked):
    g = scope[scope["pred_route_id"]==rid].dropna(subset=["latitude","longitude"]).copy()
    if g.empty: continue
    m = make_route_map_with_match(
        g, rid, color=colors[i%len(colors)],
        use_cluster=True, jitter_m=6,
        show_seq=True, dashed=True, point_style="icon"
    )
    fname = f"map_{day}_{shift}_{ttype}_{cap}seater_{rid}.html".replace(":","_")
    fpath = os.path.join(OUT_DIR, fname)
    m.save(fpath)
    html_files.append(fpath)

# 4) Index + ZIP + download
if html_files:
    index_path = os.path.join(OUT_DIR,"index.html")
    with open(index_path,"w") as f:
        f.write(f"<h2>Maps — {day} | {shift} | {ttype}</h2><ul>")
        for p in html_files:
            f.write(f'<li><a href="{os.path.basename(p)}" target="_blank">{os.path.basename(p)}</a></li>')
        f.write("</ul>")
    html_files.append(index_path)

    zip_name = f"maps_{day}_{shift}_{ttype}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.zip".replace(":","_")
    with zipfile.ZipFile(zip_name,"w",zipfile.ZIP_DEFLATED) as z:
        for p in html_files:
            z.write(p, arcname=os.path.join(os.path.basename(OUT_DIR), os.path.basename(p)))

    print("ZIP created:", zip_name)
    try:
        from google.colab import files
        files.download(zip_name)
    except:
        print("Download from left panel:", zip_name)
else:
    print("❌ No routes found with coordinates in this scope.")


Using DAY=2019-01-30, SHIFT=Day_17:30:00, TYPE=IN
Routes by seater in this scope:
 pred_capacity
4     106
6      44
12     33
Name: pred_route_id, dtype: int64
ZIP created: maps_2019-01-30_Day_17_30_00_IN_20250823_165648.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [46]:
import pandas as pd

# Pick the day (auto: busiest) or set manually
dfc = aligned_df.copy()
dfc["trip_date_str"] = pd.to_datetime(dfc["trip_date"]).dt.strftime("%Y-%m-%d")
day_str = dfc["trip_date_str"].value_counts().idxmax()
# day_str = "2019-01-26"   # uncomment to set manually

day_df = dfc[dfc["trip_date_str"] == day_str].copy()
if day_df.empty:
    raise ValueError(f"No rows for {day_str}")

# Actual = count of distinct route_no per shift (IN/OUT separate)
actual_routes = (
    day_df.groupby(["shift_key","travel_type_id"])["route_no"]
          .nunique().reset_index(name="actual_routes")
)

# Predicted = count of distinct pred_route_id per shift (IN/OUT separate)
pred_routes = (
    day_df.groupby(["shift_key","travel_type_id"])["pred_route_id"]
          .nunique().reset_index(name="predicted_routes")
)

# Merge
shift_comp = (actual_routes.merge(pred_routes,
                                  on=["shift_key","travel_type_id"],
                                  how="outer")
              .fillna(0)
              .sort_values(["shift_key","travel_type_id"]))
shift_comp["day"] = day_str
shift_comp


,shift_key,travel_type_id,actual_routes,predicted_routes,day
0,Day_08:00:00,IN,78,95,2019-01-30
1,Day_08:00:00,OUT,4,4,2019-01-30
2,Day_10:30:00,OUT,3,3,2019-01-30
3,Day_11:30:00,IN,114,148,2019-01-30
4,Day_12:30:00,IN,100,115,2019-01-30
5,Day_13:30:00,IN,79,111,2019-01-30
6,Day_14:00:00,OUT,34,41,2019-01-30
7,Day_14:30:00,IN,81,102,2019-01-30
8,Day_15:00:00,OUT,44,54,2019-01-30
9,Day_16:00:00,IN,106,143,2019-01-30


In [57]:
# After reading the raw file into df_raw
df_raw = pd.read_excel("/content/Week_routes.xlsx",sheet_name="Sheet1")  # or the path you used
# df_raw.head()

print("Raw shape:", df_raw.shape)
print("Tripsheet_no dtype:", df_raw["Tripsheet_No"].dtype)
print("Unique tripsheets overall:", df_raw["Tripsheet_No"].nunique())
print("Counts per day (raw):")
print(df_raw.groupby("Trip_Date")["Tripsheet_No"].nunique())


Raw shape: (51689, 16)
Tripsheet_no dtype: int64
Unique tripsheets overall: 12866
Counts per day (raw):
Trip_Date
2019-01-26    1117
2019-01-27     416
2019-01-28    1828
2019-01-29    2378
2019-01-30    2428
2019-01-31    2399
2019-02-01    2300
Name: Tripsheet_No, dtype: int64


In [58]:
# 🔎 Diagnose Tripsheet uniqueness and compute daily actual with a composite key if needed

import pandas as pd

# Use the same df_raw you just loaded (Sheet1)
df_raw2 = df_raw.copy()

# Normalize column names for robust handling
def norm(s): return s.strip().lower().replace(" ", "_")
df_raw2.columns = [norm(c) for c in df_raw2.columns]

# Canonical names
col_map = {
    "trip_date": ["trip_date","date","tripdate"],
    "tripsheet_no": ["tripsheet_no","tripsheet","trip_sheet_no"],
    "day_night": ["day_night","shift","daynight"],
    "trip_time": ["trip_time","time"],
    "travel_type_id": ["travel_type_id","travel_type","in_out","trip_type"],
    "facility": ["facility","plant","site"],
    "city": ["city","location"],
    "vendor": ["vendor","vendor_name","supplier"],
    "route_no": ["route_no","routeno","route"]
}

def find_col(target):
    for cand in col_map[target]:
        if cand in df_raw2.columns:
            return cand
    return None

c_trip_date   = find_col("trip_date")
c_tripsheet   = find_col("tripsheet_no")
c_day_night   = find_col("day_night")
c_trip_time   = find_col("trip_time")
c_travel_type = find_col("travel_type_id")
c_facility    = find_col("facility")
c_city        = find_col("city")
c_vendor      = find_col("vendor")
c_route_no    = find_col("route_no")

if c_trip_date is None or c_tripsheet is None:
    raise ValueError("Could not find required columns for trip_date or tripsheet_no in Sheet1.")

# Parse types safely (don’t coerce tripsheet_no to float)
df_raw2[c_trip_date] = pd.to_datetime(df_raw2[c_trip_date], errors="coerce").dt.date
if c_trip_time:
    df_raw2[c_trip_time] = pd.to_datetime(df_raw2[c_trip_time].astype(str), errors="coerce").dt.time

# Shift key from day/night + time if available
if c_day_night and c_trip_time:
    df_raw2["shift_key"] = df_raw2[c_day_night].astype(str).str.strip() + "_" + df_raw2[c_trip_time].astype(str)
elif c_day_night:
    df_raw2["shift_key"] = df_raw2[c_day_night].astype(str).str.strip()
else:
    df_raw2["shift_key"] = "NA"

# Normalize travel type to IN/OUT if present
if c_travel_type:
    df_raw2["travel_type_id"] = (df_raw2[c_travel_type].astype(str).str.upper().str.strip()
                                 .replace({"INWARD":"IN","OUTWARD":"OUT","IN-TRIP":"IN","OUT-TRIP":"OUT"}))
else:
    df_raw2["travel_type_id"] = "NA"

# --- 1) Check duplicates of Tripsheet across possible disambiguators
cand_keys = [k for k in [c_facility, c_city, c_vendor, c_route_no] if k is not None]

dups_report = {}
for key in cand_keys:
    # Number of tripsheet numbers that appear under multiple values of this key
    tmp = (df_raw2.groupby([c_tripsheet, key]).size().reset_index(name="n"))
    counts_per_tripsheet = tmp.groupby(c_tripsheet)[key].nunique()
    multi = (counts_per_tripsheet > 1).sum()
    dups_report[key] = int(multi)

print("Possible disambiguators and how many Tripsheet_No repeat across them:")
for k, v in dups_report.items():
    print(f" - {k}: {v} tripsheets span multiple values")

# Show a few examples if any repeats
for k, v in dups_report.items():
    if v > 0:
        print(f"\nExamples where Tripsheet_No spans multiple '{k}' values:")
        tmp = (df_raw2.groupby([c_tripsheet, k]).size().reset_index(name="n"))
        counts = tmp.groupby(c_tripsheet)[k].nunique()
        bad_ids = counts[counts > 1].index.tolist()[:5]
        display(tmp[tmp[c_tripsheet].isin(bad_ids)].sort_values([c_tripsheet, k]).head(20))
        break

# --- 2) Daily actual routes, two ways

base = df_raw2.dropna(subset=[c_trip_date, c_tripsheet]).copy()

# V1: simple (unique tripsheet per day/shift/type)
actual_v1 = (
    base.groupby([c_trip_date, "shift_key", "travel_type_id"])[c_tripsheet]
        .nunique().reset_index(name="actual_routes_v1")
    .groupby(c_trip_date)["actual_routes_v1"].sum().reset_index()
)

# V2: composite key adds any available disambiguators where tripsheets repeat
composite_cols = [c_tripsheet]
for k, v in dups_report.items():
    if v > 0:
        composite_cols.append(k)

actual_v2 = (
    base.groupby([c_trip_date, "shift_key", "travel_type_id"] + composite_cols)
        .size().reset_index(name="n")  # each unique combo = one actual route
    .groupby(c_trip_date).size().reset_index(name="actual_routes_v2")
)

# Merge V1 & V2 to compare
actual_compare = actual_v1.merge(actual_v2, on=c_trip_date, how="left").rename(columns={c_trip_date:"trip_date"})
actual_compare = actual_compare.sort_values("trip_date")
print("\nDaily actual routes — simple vs composite key:")
display(actual_compare)

# If you have aligned_df (predictions), compare against the composite (more accurate)
if 'aligned_df' in globals():
    pred_daily = (
        aligned_df.groupby(["trip_date","shift_key","travel_type_id"])["pred_tripsheet_no"]
                  .nunique().reset_index(name="predicted_routes")
        .groupby("trip_date")["predicted_routes"].sum().reset_index()
    )
    comp = actual_compare.merge(pred_daily, on="trip_date", how="left")
    comp["extra_routes_vs_v2"] = comp["predicted_routes"] - comp["actual_routes_v2"]
    comp["pct_increase_vs_v2"] = 100 * comp["extra_routes_vs_v2"] / comp["actual_routes_v2"]
    print("\nDaily Actual (composite) vs Predicted:")
    display(comp)


/tmp/ipython-input-1260452304.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_raw2[c_trip_time] = pd.to_datetime(df_raw2[c_trip_time].astype(str), errors="coerce").dt.time


Possible disambiguators and how many Tripsheet_No repeat across them:
 - facility: 0 tripsheets span multiple values
 - city: 0 tripsheets span multiple values
 - route_no: 0 tripsheets span multiple values

Daily actual routes — simple vs composite key:


,trip_date,actual_routes_v1,actual_routes_v2
0,2019-01-26,1117,1117
1,2019-01-27,416,416
2,2019-01-28,1828,1828
3,2019-01-29,2378,2378
4,2019-01-30,2428,2428
5,2019-01-31,2399,2399
6,2019-02-01,2300,2300



Daily Actual (composite) vs Predicted:


,trip_date,actual_routes_v1,actual_routes_v2,predicted_routes,extra_routes_vs_v2,pct_increase_vs_v2
0,2019-01-26,1117,1117,1297,180,16.114593
1,2019-01-27,416,416,508,92,22.115385
2,2019-01-28,1828,1828,2241,413,22.592998
3,2019-01-29,2378,2378,2892,514,21.614802
4,2019-01-30,2428,2428,2936,508,20.922570
5,2019-01-31,2399,2399,2886,487,20.300125
6,2019-02-01,2300,2300,2831,531,23.086957
